# S_Game_Details

In [12]:
import pandas as pd
from datetime import datetime
import sqlite3
import hashlib
import ssl
from urllib.error import HTTPError
ssl._create_default_https_context = ssl._create_unverified_context

In [13]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name_1 = "Source_S_Game_Details"
table_name_2='S_Game_Details'
connection=sqlite3.connect(database=data_base)
cursor = connection.cursor()
drop_query_1 = f"DROP TABLE IF EXISTS {table_name_1};"
drop_query_2 = f"DROP TABLE IF EXISTS {table_name_2};"
cursor.execute(drop_query_1)
cursor.execute(drop_query_2)
connection.commit()
connection.close()

Get Details from "D:/AFL_GRP/DataBase/AFLStats.db"."GamesPlayed" and load into "D:/AFL_GRP/DataBase/AFLDataVault.db"."Source_S_Game_Details"

In [14]:
data_base=f'D:/AFL_GRP/DataBase/AFLStats.db'
connection=sqlite3.connect(data_base)
query=f''' 
        select
        *
        from GamesPlayed 
        where 1=1
       
    '''
df=pd.read_sql(query,connection)
connection.close()

data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='Source_S_Game_Details'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()


Check if we need to fix drawn Grand Final i 2010 replace Roun from 'GF' to 'GF1'

In [15]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query = f''' 
select * from Source_S_Game_Details where Year=2010 and Round='GF1'
'''

df=pd.read_sql(query,connection)
connection.close()
df

,Year,Round,Team,Opponent,Date_Played,Ground,Team_Goals,Team_Behind,Opponent_Goals,Opponent_Behind,Team_Result,Opponent_Result
0,2010,GF1,CW,SK,2010-09-25 00:00:00,M.C.G.,9,14,10,8,9,9


Get the hashkey fro game from H_Game

In [16]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
connection=sqlite3.connect(database=data_base)
query=f'''  
select  hg.Game_hashkey
        ,ssgd.date_played
        ,ssgd.Ground
        ,ssgd.Team_Goals
        ,ssgd.Team_behind 
        ,ssgd.Opponent_Goals
        ,ssgd.Opponent_Behind
        ,ssgd.Team_Result
        ,ssgd.Opponent_Result
from    Source_S_Game_Details ssgd
join    H_Game hg
on      ssgd.Year=hg.year
and     ssgd.Round=hg.round
and     ssgd.Team=hg.team
and     ssgd.Opponent=hg.Opponent
'''
df=pd.read_sql(query,connection)
connection.close()
df

,Game_Hashkey,Date_Played,Ground,Team_Goals,Team_Behind,Opponent_Goals,Opponent_Behind,Team_Result,Opponent_Result
0,940baad6e3b1f642ebd20ef7f2f08033a4c38ec9633a4e...,1897-05-08 00:00:00,Brunswick St,6,13,2,4,1,0
1,596615ac0b4e8508c5cbf35782f6205f35b1fe87592f40...,1897-05-08 00:00:00,Victoria Park,5,11,2,4,1,0
2,a79f663d1f61f8909f15adebce2ff417fbe205cde73112...,1897-05-08 00:00:00,Corio Oval,3,6,7,5,0,1
3,94c731f7c2d3e4e99889d206d252c5492c679117c8bda1...,1897-05-08 00:00:00,Lake Oval,3,9,6,8,0,1
4,c5b1319e8e8bdc2f7024068131b81857811b5768cbb777...,1897-05-15 00:00:00,Lake Oval,6,4,5,6,1,0
...,...,...,...,...,...,...,...,...,...
16631,4e36b57e6c268c3dea3e52cbaa32d906ec9d44b1e14886...,2024-09-13 00:00:00,Adelaide Oval,11,9,11,6,1,0
16632,24f70b6581fd27418323e1e9a6d176ed1df6f4e3cf9b65...,2024-09-14 00:00:00,Sydney Showground,15,10,15,15,0,1
16633,ec692efb8fa838799813568da7c9014f816ef21679fdfc...,2024-09-20 00:00:00,S.C.G.,14,11,8,11,1,0
16634,2da19cdc91d7ddf46e58c77f4e17624878c3ccf6a0ed52...,2024-09-21 00:00:00,M.C.G.,12,13,14,11,0,1


In [17]:



def hash_diff(row):
    # Combine the values from the correct column names, converting them to strings
    combined = (str(row['Date_Played']) + str(row['Ground']) + str(row['Team_Goals']) + 
                str(row['Team_Behind']) + str(row['Opponent_Goals']) + 
                str(row['Opponent_Behind']) + str(row['Team_Result']) + 
                str(row['Opponent_Result']))
    
    # Generate a hash using SHA-256
    return hashlib.sha256(combined.encode()).hexdigest()

# Assuming your DataFrame is called df, apply the hash function
df['HashDiff'] = df.apply(hash_diff, axis=1)

# Add the Record_Source column
df['Record_Source'] = 'ScrapedPlayerStatistics'

# Ensure all column names are correct in the DataFrame selection
df = df[['Game_Hashkey', 'Date_Played', 'Ground', 'Team_Goals', 'Team_Behind', 
         'Opponent_Goals', 'Opponent_Behind', 'Team_Result', 
         'Opponent_Result', 'HashDiff', 'Record_Source']]

# Display the updated DataFrame


In [18]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'
table_name='S_Game_Details'
connection=sqlite3.connect(database=data_base)
df.to_sql(table_name, connection, index=False, if_exists='append')
connection.close()


Test Hub Sat link

In [19]:
data_base=f'D:/AFL_GRP/DataBase/AFLDataVault.db'

connection=sqlite3.connect(database=data_base)
query =f'''
select  hg.Year, hg.Round,hg.Team,hg.Opponent,sgd.Team_Result,sgd.Opponent_Result 
       
from h_game hg
join S_Game_details sgd
on   hg.Game_hashkey=sgd.game_hashkey
where hg.year=2024
and hg.round='GF'

'''
df=pd.read_sql(query,connection)
connection.close()
df

,Year,Round,Team,Opponent,Team_Result,Opponent_Result
0,2024,GF,SW,BL,0,1
